<font color="#CA3532"><h1 align="left">Master Data Analytics. EDEM.</h1></font>
<font color="#6E6E6E"><h2 align="left">Herramientas MLOps.</h2></font> 
<font color="#6E6E6E"><h2 align="left">Tarea 1. Pipeline entrenamiento de modelos.</h2></font> 
#### Daniel Ruiz Riquelme
https://docs.metaflow.org/metaflow/basics

##  Install dependencies

In [13]:
#!pip install metaflow
#!pip install scikit-learn
#!pip install pandas

## Set username

In [14]:
# Set username for workflows
import os
os.environ["USERNAME"] = "nachoreyes"

In [17]:
%%writefile metaflow_trainingflow.py
from metaflow import FlowSpec, Parameter, step
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
import pickle

class TrainingFlow(FlowSpec):
    max_depth = Parameter('max_depth', default=2, help='Max depth of the random forest classifier')
    n_estimators = Parameter('n_estimators', default=100, help='Number of estimators for the random forest classifier')
    random_state = Parameter('random_state', default=0, help='Random state for the random forest classifier')
    
    @step
    def start(self):
        self.next(self.ingest_data)
        
    @step
    def ingest_data(self):
        iris = load_iris()
        self.X = iris.data
        self.y = iris.target
        self.next(self.split_data)

    @step
    def split_data(self):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2)
        self.next(self.train)

    @step
    def train(self):
        self.clf = RandomForestClassifier(max_depth=self.max_depth, n_estimators=self.n_estimators, random_state=self.random_state)
        self.clf.fit(self.X_train, self.y_train)
        self.next(self.show_metrics)

    @step
    def show_metrics(self):
        self.y_pred = self.clf.predict(self.X_test)
        print(confusion_matrix(self.y_test, self.y_pred))
        print(classification_report(self.y_test, self.y_pred))
        self.next(self.register_model)
        
    @step
    def register_model(self):
        with open('model_second_day.pkl', 'wb') as f:
            pickle.dump(self.clf, f)
        self.next(self.end)
        
    @step
    def end(self):
        pass
    
if __name__ == '__main__':
    TrainingFlow()


Overwriting metaflow_trainingflow.py


In [18]:
!python3 metaflow_trainingflow.py run --max_depth 2 --n_estimators 100 --random_state 0

Metaflow 2.11.16 executing TrainingFlow for user:nachoreyes
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
2024-05-22 16:44:31.326 Workflow starting (run-id 1716389071325121):
2024-05-22 16:44:31.335 [1716389071325121/start/1 (pid 5398)] Task is starting.
2024-05-22 16:44:31.991 [1716389071325121/start/1 (pid 5398)] Task finished successfully.
2024-05-22 16:44:31.996 [1716389071325121/ingest_data/2 (pid 5404)] Task is starting.
2024-05-22 16:44:32.686 [1716389071325121/ingest_data/2 (pid 5404)] Task finished successfully.
2024-05-22 16:44:32.690 [1716389071325121/split_data/3 (pid 5407)] Task is starting.
2024-05-22 16:44:33.348 [1716389071325121/split_data/3 (pid 5407)] Task finished successfully.
2024-05-22 16:44:33.352 [1716389071325121/train/4 (pid 5410)] Task is starting.
2024-05-22 16:44:34.036 [1716389071325121/train/4 (pid 5410)] Task finished successfully.
2024-05-22 16:44:34.041 [1716389071325121/show_me